In [1]:
from desc.objectives import ParticleTracer, ObjectiveFunction
from desc.examples import get
from desc.grid import Grid
import desc.io
from desc.backend import jnp
import matplotlib.pyplot as plt
import numpy as np

DESC version 0+untagged.5523.g890c640.dirty,using JAX backend, jax version=0.4.14, jaxlib version=0.4.14, dtype=float64
Using device: CPU, with 2.16 GB available memory


In [2]:
eq = desc.io.load("../equilibria/test_run.h5")
eq._iota = eq.get_profile("iota").to_powerseries(order=eq.L, sym=True)
eq._current = None
# eq.solve()

In [3]:
tmin = 0
tmax = 0.00007
nt = 20
time = jnp.linspace(tmin, tmax, nt)
psi_i = 0.7
theta_i = 0.2
zeta_i = 0.2

mass = 1.673e-27
Energy = 3.52e6*1.6e-19
ini_vpar = 0.5*jnp.sqrt(2*Energy/mass)
ini_cond = [psi_i, theta_i, zeta_i, float(ini_vpar)]

mass_charge = mass/1.6e-19

grid = Grid(jnp.array([jnp.sqrt(psi_i), theta_i, zeta_i]).T, jitable=True, sort=False)
data = eq.compute("|B|", grid=grid)

mu = Energy/(mass*data["|B|"]) - (ini_vpar**2)/(2*data["|B|"])

ini_param = (float(mu), mass_charge)


objective = ParticleTracer(eq=eq, output_time=time, initial_conditions=ini_cond, initial_parameters=ini_param)

In [4]:
objective.build()
solution = objective.compute(*objective.xs(eq))
print(solution)

2023-09-07 16:04:30.534917: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_diffeqsolve] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-09-07 16:05:55.581195: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 3m25.046379907s

********************************
[Compiling module jit_diffeqsolve] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


[ 0.         -0.07586199 -0.18096794 -0.20286074 -0.11753264 -0.01750423
 -0.01744451 -0.1167273  -0.19979871 -0.18054003 -0.07852638 -0.0029902
 -0.04138598 -0.15351753 -0.21014469 -0.15238247 -0.04210961  0.00113745
 -0.07078478 -0.17916198]


In [5]:
objective = ParticleTracer(eq=eq, output_time=time, initial_conditions=ini_cond, initial_parameters=ini_param)
ObjFunction = ObjectiveFunction([objective])
ObjFunction.build()


Building objective: Particle Tracer


In [6]:
ObjFunction.compile()

Compiling objective function and derivatives: ['Particle Tracer']

2023-09-07 16:09:24.376862: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 2m17.670373973s

********************************
[Compiling module jit_compute_scalar] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


In [ ]:
ObjFunction.grad(ObjFunction.x(eq))
#print(ObjFunction.x(eq))
objective.xs(eq)


In [ ]:
objective.args